In [ ]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

In [ ]:
ci_key = os.environ.get("CARBON_INTERFACE_KEY")
base_url = "https://www.carboninterface.com/api/v1"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {ci_key}",
}


endpoint = f"{base_url}/estimates"
body = {
        "type": "flight",
        "passengers": 1,
        "legs": [
            {"departure_airport": "LHR", "destination_airport": "BCN"},
            {"departure_airport": "GOT", "destination_airport": "BCN"}
        ],
    }
request = requests.post(endpoint, headers=headers, json=body)
request.status_code
request.text

In [ ]:
trips = pd.Series(
            {"departure_airport": "GOT, LHR, CDG"})
trips

In [ ]:
for departure in trips.departure_airport.split(","):
    print(departure.strip())


In [ ]:
legs = []
for departure in trips.departure_airport.split(","):
    legs.append(
        {
            "departure_airport": departure.strip(),
            "destination_airport": "BCN",
        }
    )
body = {"type": "flight", "passengers": 1, "legs": legs}
print(body)


In [ ]:
request = requests.post(endpoint, json=body, headers=headers)

In [ ]:
print(request.text)

In [ ]:
data = request.json()["data"]["attributes"]
trip_emissions = [{"CO2 (kg)": data["carbon_kg"], "CO2 (mt)": data["carbon_mt"]}]
trip_emissions

In [ ]:
pd.DataFrame.from_dict(trip_emissions)

In [ ]:
request.text

In [ ]:
data = request.json()["data"]["attributes"]
trip_emissions = {"CO2 (kg)": data["carbon_kg"],
                  "CO2 (mt)": data["carbon_mt"]}
trip_emissions

In [ ]:
leg1 = {
    "data": {
        "id": "fadb29cf-50bc-4369-9a33-152c5007dfc0",
        "type": "estimate",
        "attributes": {
            "passengers": 1,
            "legs": [{"departure_airport": "GOT", "destination_airport": "BCN"}],
            "distance_value": 2058.09,
            "distance_unit": "km",
            "estimated_at": "2024-04-30T13:25:12.342Z",
            "carbon_g": 118392,
            "carbon_lb": 261.01,
            "carbon_kg": 118.39,
            "carbon_mt": 0.12,
        },
    }
}
leg2 = {
    "data": {
        "id": "22919fe5-110d-4321-87fa-c4074edeacd7",
        "type": "estimate",
        "attributes": {
            "passengers": 1,
            "legs": [{"departure_airport": "LHR", "destination_airport": "BCN"}],
            "distance_value": 1247.49,
            "distance_unit": "km",
            "estimated_at": "2024-04-30T13:27:00.387Z",
            "carbon_g": 91822,
            "carbon_lb": 202.43,
            "carbon_kg": 91.82,
            "carbon_mt": 0.09,
        },
    }
}
legs_combined = {
    "data": {
        "id": "f8128e5d-1edf-44f5-83aa-7637e3b56b7e",
        "type": "estimate",
        "attributes": {
            "passengers": 1,
            "legs": [
                {"departure_airport": "LHR", "destination_airport": "BCN"},
                {"departure_airport": "GOT", "destination_airport": "BCN"},
            ],
            "distance_value": 3305.58,
            "distance_unit": "km",
            "estimated_at": "2024-04-30T13:27:34.565Z",
            "carbon_g": 210215,
            "carbon_lb": 463.45,
            "carbon_kg": 210.22,
            "carbon_mt": 0.21,
        },
    }
}

In [ ]:
flights = pd.read_csv("flights_csv.csv").drop(columns="Unnamed: 0")
flights.head(20)

In [ ]:
# The below is not a great idea since it disconnets the row values from each other
cheapest_per_combo = flights.groupby(["airport_from", "airport_to"]).agg("min")
cheapest_per_combo.head(1)

In [ ]:
cheapest_combos = (
    flights.sort_values(by="price", ascending=True)
    .groupby(["city_from", "city_to"])
    .agg("first")
).reset_index()
cheapest_combos.sort_values(by=["city_from", "city_to"])

In [ ]:
group_destinations = (
    cheapest_combos.groupby(["city_to"])
    .agg({"price": "sum"})
    .sort_values("price")
    .reset_index()
)
group_destinations

In [ ]:
cheapest_group_destination = group_destinations.iloc[0]
print(
    f'The cheapetst destination for you and your friends to meet is {cheapest_group_destination["city_to"]}, '
    f'at €{cheapest_group_destination["price"]} in total or €{round((cheapest_group_destination["price"]/3),2)} per person.'
)

In [ ]:
cheapest_combos.loc[cheapest_combos["city_to"] == cheapest_group_destination["city_to"]]

In [ ]:
load_dotenv()
kiwi_user = os.environ.get("KIWI_USER")
kiwi_key = os.environ.get("KIWI_KEY")
base_url = "https://api.tequila.kiwi.com"
headers = {"Content-Type": "application/json", "apikey": kiwi_key}

In [ ]:
params = {
    "term": "",
    "limit": 500,
    "sort": "name",
    "active_only": True,
    "source_popularity": "bookings",
}
origins = ["gothenburg_se", "barcelona_es", "paris_fr", "berlin_de"]
for i, city in enumerate(origins):
    params["term"] = city
    res = requests.get(
        f"{base_url}/locations/topdestinations", params=params, headers=headers
    )
    if res.status_code == 200:
        data = res.json()
        print(f"{data['results_retrieved']} results retrieved for {city}")
        retrieved_locations = {location["id"] for location in data["locations"]}
        retrieved_locations.add(city)
        if i == 0:
            all_results = retrieved_locations
        else:
            all_results = all_results.intersection(retrieved_locations)
    else:
        print(f"Something went wrong for {city}! Status: {res.status_code}")

In [ ]:
print("Total number of common destinations:", len(all_results))
print(all_results)

In [ ]:
endpoint = f"{base_url}/locations"
query = f"/query?term=London"
response = requests.get(url=endpoint + query, headers=headers)
response.raise_for_status()
city_data = response.json()
print(response.status_code)
print(city_data)

In [ ]:
import datetime as dt

endpoint = f"{base_url}/v2/search?query="
max_price = 1000
today = dt.datetime.today()
months_from_today = today + dt.timedelta(days=30)
# from_date = self.today.strftime("%d/%m/%Y")
# to_date = self.months_from_today.strftime("%d/%m/%Y")
# stopovers = int(city_data["stopovers"])
params = {
    "fly_from": "paris_fr",
    "fly_to": "berlin_de",
    "date_from": (today + dt.timedelta(days=7)).strftime("%d/%m/%Y"),
    "date_to": (today + dt.timedelta(days=14)).strftime("%d/%m/%Y"),
    "price_to": max_price,
    "nights_in_dst_from": 3,
    "nights_in_dst_to": 10,
    "one_for_city": True,
    "one_per_date": True,
    "curr": "EUR",
    "max_stopovers": 0,
    "limit": 500,
}
response = requests.get(url=endpoint, params=params, headers=headers)
response.raise_for_status()
data = response.json()["data"]

In [ ]:
len(data)

In [ ]:
print(response.json())

In [ ]:
data

for d in data:
    print(d["cityTo"])

In [ ]:

    def __init__(self):
        load_dotenv()
        self.ci_key = os.environ.get("CARBON_INTERFACE_KEY")
        self.base_url = "https://www.carboninterface.com/api/v1"
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.ci_key}",
        }
        self.today = dt.datetime.today()
        self.months_from_today = self.today + dt.timedelta(days=30)

    def estimate_flight_emissions(self, departure, destination):
        endpoint = f"{self.base_url}/estimates"
        body = {
            "type": "flight",
            "passengers": 1,
            "legs": [
                {"departure_airport": departure, "destination_airport": destination},
            ],
        }
        request = requests.post(endpoint, json=body)